### 3.4.1. 分类问题
独热编码（one-hot encoding）。 独热编码是一个向量，它的分量和类别一样多。 类别对应的分量设置为1，其他所有分量设置为0。

### 3.4.4. softmax运算
要将输出视为概率，我们必须保证在任何数据上的输出都是非负的且总和为1。 此外，我们需要一个训练的目标函数，来激励模型精准地估计概率。 例如， 在分类器输出0.5的所有样本中，我们希望这些样本是刚好有一半实际上属于预测的类别。 这个属性叫做校准（calibration）。  
社会科学家邓肯·卢斯于1959年在选择模型（choice model）的理论基础上 发明的softmax函数正是这样做的： softmax函数能够将未规范化的预测变换为非负数并且总和为1，同时让模型保持 可导的性质。 为了完成这一目标，我们首先对每个未规范化的预测求幂，这样可以确保输出非负。 为了确保最终输出的概率值总和为1，我们再让每个求幂后的结果除以它们的总和。如下式：
$$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o})\quad \text{其中}\quad \hat{y}_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)}$$
只会确定分配给每个类别的概率。 因此，在预测过程中，我们仍然可以用下式来选择最有可能的类别。  
$$\operatorname*{argmax}_j \hat y_j = \operatorname*{argmax}_j o_j.$$
尽管softmax是一个非线性函数，但softmax回归的输出仍然由输入特征的仿射变换决定。 因此，softmax回归是一个线性模型（linear model）。

### 3.4.6. 损失函数
cross-entropy loss:$$l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j.$$
softmax和cross-cntropy loss结合及其求导:
$$\begin{split}\begin{aligned}
l(\mathbf{y}, \hat{\mathbf{y}}) &=  - \sum_{j=1}^q y_j \log \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} \\
&= \sum_{j=1}^q y_j \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j\\
&= \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j.
\end{aligned}\end{split}$$
考虑相对于任何未规范化的预测$o_{j}$的导数，我们得到：$$\partial_{o_j} l(\mathbf{y}, \hat{\mathbf{y}}) = \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} - y_j = \mathrm{softmax}(\mathbf{o})_j - y_j.$$
换句话说，导数是我们softmax模型分配的概率与实际发生的情况（由独热标签向量表示）之间的差异。 从这个意义上讲，这与我们在回归中看到的非常相似， 其中梯度是观测值$y$和估计值$\hat{y}$之间的差异。 这不是巧合，在任何指数族分布模型中 （参见本书附录中关于数学分布的一节）， 对数似然的梯度正是由此得出的。 这使梯度计算在实践中变得容易很多。

### 3.4.7. 信息论基础
熵：$$H[P] = \sum_j - P(j) \log P(j).$$  其中P为分布函数  
信息量：$$\log \frac{1}{P(j)} = -\log P(j)$$
如果把熵$H(P)$想象为“知道真实概率的人所经历的惊异程度”，那么什么是交叉熵？ 交叉熵从$P$
到$Q$，记为$H(P,Q)$。 我们可以把交叉熵想象为“主观概率为$Q$的观察者在看到根据概率$P$生成的数据时的预期惊异”。 当$P=Q$时，交叉熵达到最低。 在这种情况下，从$P$到$Q$的交叉熵是$H(P,P) = H(P)$。

简而言之，我们可以从两方面来考虑交叉熵分类目标： （i）最大化观测数据的似然；（ii）最小化传达标签所需的惊异。

### Practice
### 我们可以更深入地探讨指数量化与 softmax 之间的联系。  

1. **计算 softmax 交叉熵损失 $L(y, \hat{y})$ 的二阶导数：**  
   - 首先，计算 softmax 函数的输出 $o$，并构造交叉熵损失 $L$：  
     
     $$  
     L(y, \hat{y}) = -\sum_{i} y_i \log(\hat{y_i})  
     $$  
     
   - 然后，计算 softmax 输出 $o$ 的分布并与交叉熵损失的二阶导数匹配。可以通过链式法则进行推导。  

### 假设我们有三个类别的概率相等，即概率量是 $(\frac{1}{3}, \frac{1}{3}, \frac{1}{3})$：  

1. **如果我们尝试为其设计随机代码，有何问题？**  
   - 问题在于，简单的随机生成可能不会反映类别的真实概率分布，容易导致不平衡的输出。  

2. **请设计一个更好的代码，示范：如果我们尝试编码两个独立的观察结果会发生什么？如果我们联结代码的预测值怎么办？**  
   - 可以使用 Monte Carlo 方法针对每个观察结果生成样本，并对采样结果进行平均，以确保生成的类别分布尽可能接近真实概率。  

### softmax 是上面介绍的映射的误称（尽管深度学习领域很多人都使用这个名词）。真正的 softmax 被定义为：  

$$  
\text{RealSoftMax}(a, b) = \log(\exp(a) + \exp(b))  
$$  

1. **证明 $\text{RealSoftMax}(a, b) > \max(a, b)$。**  
   - 通过考察 $\exp(a)$ 和 $\exp(b)$ 的性质，可以证明 $\log(\exp(a) + \exp(b))$ 总是大于任何一个输入值。  

2. **证明 $a^{-1}\text{RealSoftMax}(\lambda_a, \lambda_b) > \max(\lambda_a, \lambda_b)$ 成立，前提是 $a > 0$。**  
   - 利用 log 和指数的单调性，结合 $\lambda_a, \lambda_b$ 的性质，可以得出这个结论。  

3. **证明对于 $\lambda \to \infty$，有 $1/\lambda \text{RealSoftMax}(\lambda_a, \lambda_b) \to \max(a, b)$。**  
   - 当 $\lambda$ 趋近于无穷大时，主要成分会主导表达式，从而导致结果趋向于 $\max(a, b)$。  

4. **soft-min 会有什么性质？**  
   - soft-min 通常是 softmax 的对偶运算，会允许对最小值进行类似的概率准备。  

5. **将其扩展到两个以上的数字。**  
   - soft-max 和 soft-min 的概念可以直接扩展到更多类别，只需对 exponentials 和 logs 进行适当的处理。